# Advanced Topics

The information beyond this section provides a brief introduction to advanced topics with the `Spatial DataFrame` structure.  

## Spatial Indexing
One of the most important tasks for any computer system is to quickly retrieve information. Enterprise systems, whether storing GIS information or not, all utilize the concept of indexing to quickly search through large data stores.  Rather than searching every an entire dataset or every row and column of a table, indexes partition tables into smaller structures that allow for fast and random access. Indexing methods for standard data types like numbers, strings and dates make use of sort orders to search through data. Spatial indexes are based on the concept of a minimum bounding rectangle - the smallest rectangle that contains the entire geometric shape. Querying rectangles is an inexpensive operation when analyzing geometries relative to querying an entire feature array composed of numerous coordinate pairs. Complex lines and irregularly-shaped polygons can easily be queried to analyze relationships between features through different kinds of spatial indexes.   

The Spatial DataFrame uses an implementation of spatial indexing known as [QuadTree indexing](https://en.wikipedia.org/wiki/Quadtree), which breaks down a dataset into nodes that have zero or four children and searches these nodes when determining locations, relationships and attributes of specific features. In the [**Examining Feature Layer content**](#Example:-Examining-Feature-Layer-content) section of this notebook, the USA Major Cities feature layer was queried and the `df` method was called on the results to create a data frame. The [`sindex`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.html?highlight=style#arcgis.features.SpatialDataFrame.sindex) method on the `df` creates a quad tree index.

The `intersect` method of the resulting index takes a bounding box as input (4 coordinates representing the minimum and maximum x,y coordinate pairs) and returns a list of features that intersect that bounding box.

* [Advanced Topics](#Advanced-Topics)
 * [Spatial Indexing](#Spatial-Indexing)
 * [Spatial Joins](#Spatial-Joins)
   * [Example: Merging State Statistics with Cities](#Example:-Merging-State-Statistics-Information-with-Cities)

In [2]:
from arcgis.gis import GIS

In [3]:
from arcgis import GIS
item = GIS().content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")
flayer = item.layers[0]
df = flayer.query(where="AGE_45_54 < 1500").df
df.head()

,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_5_9,AGE_65_74,AGE_75_84,...,PLACEFIPS,POP2010,POPULATION,POP_CLASS,RENTER_OCC,ST,STFIPS,VACANT,WHITE,SHAPE
0,1413,1381,1106,2138,1815,1411,979,1557,525,307,...,0468080,14287,14980,6,1074,AZ,04,261,9196,"{'x': -12768343.256613126, 'y': 3842463.708135..."
1,727,738,677,1380,1185,1333,1087,740,661,444,...,0602042,9932,10239,6,2056,CA,06,267,8273,"{'x': -13613950.337588644, 'y': 4931686.754090..."
2,593,511,2323,2767,746,127,34,1229,4,2,...,0610561,10616,11869,6,2558,CA,06,296,7530,"{'x': -13066582.116550362, 'y': 3925650.676616..."
3,888,988,900,1729,1479,1443,959,766,514,280,...,0613560,10866,11195,6,761,CA,06,86,5898,"{'x': -13123874.446103057, 'y': 4044249.710416..."
4,1086,1228,1013,1822,1759,1478,1112,925,687,477,...,0614974,12823,13009,6,1763,CA,06,88,6930,"{'x': -13151212.145276317, 'y': 4027601.332347..."


In [7]:
df.idxmax?

In [3]:
si = df.sindex
index = si.intersect([-13043219.122301877, 3911134.034258818, 
                      -13243219.102301877, 4111134.0542588173])

In [4]:
df.iloc[index]

,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_5_9,AGE_65_74,AGE_75_84,...,PLACEFIPS,POP2010,POPULATION,POP_CLASS,RENTER_OCC,ST,STFIPS,VACANT,WHITE,SHAPE
2,593,511,2323,2767,746,127,34,1229,4,2,...,0610561,10616,11869,6,2558,CA,06,296,7530,"{'x': -13066582.116550362, 'y': 3925650.676616..."
3,888,988,900,1729,1479,1443,959,766,514,280,...,0613560,10866,11195,6,761,CA,06,86,5898,"{'x': -13123874.446103057, 'y': 4044249.710416..."
4,1086,1228,1013,1822,1759,1478,1112,925,687,477,...,0614974,12823,13009,6,1763,CA,06,88,6930,"{'x': -13151212.145276317, 'y': 4027601.332347..."
10,1076,1118,952,1707,1651,1397,931,955,524,258,...,0634302,11570,12025,6,808,CA,06,105,5275,"{'x': -13080593.062843386, 'y': 4012557.378155..."
14,14,24,44,113,153,523,2425,11,3883,5122,...,0639259,16192,17499,6,2572,CA,06,1714,14133,"{'x': -13105436.040130444, 'y': 3977081.952377..."
22,1041,1219,1041,1402,1317,1271,727,996,396,145,...,0650132,10644,11092,6,963,CA,06,212,4459,"{'x': -13063142.772085657, 'y': 4049606.978283..."


## Spatial Joins

A Spatial join is a GIS operation that affixes data from one feature layer’s attribute table to another based on a spatial relationship.

The spatial join involves matching rows from the Join Features (data frame1) to the Target Features (data frame2) based on their relative spatial locations.  

#### Example: Merging State Statistics Information with Cities

The goal is to get Wyoming's city locations and census data joined with Wymoing's state census data.
> If you do not access to the `ArcPy` site-package from the Python interpreter used to execute the following cells, you must authenticate to an ArcGIS Online Organization or ArcGIS Enterprise portal.

> g3 = GIS("https://www.arcgis.com", "username", "password")

In [5]:
g2 = GIS("https://pythonapi.playground.esri.com/portal", "arcgis_python", "amazing_arcgis_123")

In [8]:
from arcgis.features import SpatialDataFrame

In [6]:
import os
#data_pth = r'/path/to/your/data/census_2010/example'
data_pth = r"/Volumes/Data/My_Projects/Python_API/notebooks/data/census_2010/"
cities = r"cities.shp"
states = r"states.shp"

In [9]:
sdf_target = SpatialDataFrame.from_featureclass(os.path.join(data_pth, cities))
sdf_join = SpatialDataFrame.from_featureclass(os.path.join(data_pth, states))

We will use python's list comprehensions to create lists of the attribute columns in the dataframe, then print out the lists to see the names of all the attribute columns.

In [10]:
sdf_target_cols = [column for column in sdf_target.columns]
sdf_join_cols = [column for column in sdf_join.columns]

Print out a list of columns in the `sdf_target` dataframe created from the cities shapefile:

In [11]:
for a,b,c,d in zip(sdf_target_cols[::4],sdf_target_cols[1::4],sdf_target_cols[2::4], sdf_target_cols[3::4]):
    print("{:<30}{:<30}{:<30}{:<}".format(a,b,c,d))

index                         AGE_18_21                     AGE_22_29                     AGE_30_39
AGE_40_49                     AGE_50_64                     AGE_5_17                      AGE_65_UP
AGE_UNDER5                    AMERI_ES                      AREALAND                      AREAWATER
ASIAN                         AVE_FAM_SZ                    AVE_HH_SZ                     BLACK
CAPITAL                       CLASS                         FAMILIES                      FEMALES
FHH_CHILD                     HAWN_PI                       HISPANIC                      HOUSEHOLDS
HSEHLD_1_F                    HSEHLD_1_M                    HSE_UNITS                     MALES
MARHH_CHD                     MARHH_NO_C                    MED_AGE                       MED_AGE_F
MED_AGE_M                     MHH_CHILD                     MULT_RACE                     NAME
OTHER                         OWNER_OCC                     PLACEFIP                      POP2000
POP2007         

Print out a list of columns in the `sdf_join` dataframe created from the states shapefile:

In [12]:
for a,b,c,d,e in zip(sdf_join_cols[::5],sdf_join_cols[1::5],sdf_join_cols[2::5],sdf_join_cols[3::5],sdf_join_cols[4::5]):
    print("{:<20}{:<20}{:<20}{:<20}{:<}".format(a,b,c,d,e))

index               AGE_18_21           AGE_22_29           AGE_30_39           AGE_40_49
AGE_50_64           AGE_5_17            AGE_65_UP           AGE_UNDER5          AMERI_ES
ASIAN               AVE_FAM_SZ          AVE_HH_SZ           AVG_SALE07          AVG_SIZE07
BLACK               CROP_ACR07          FAMILIES            FEMALES             FHH_CHILD
HAWN_PI             HISPANIC            HOUSEHOLDS          HSEHLD_1_F          HSEHLD_1_M
HSE_UNITS           MALES               MARHH_CHD           MARHH_NO_C          MED_AGE
MED_AGE_F           MED_AGE_M           MHH_CHILD           MULT_RACE           NO_FARMS07
OTHER               OWNER_OCC           POP00_SQMI          POP10_SQMI          POP2000
POP2010             RENTER_OCC          SHAPE               SQMI                STATE_ABBR
STATE_FIPS          STATE_NAME          SUB_REGION          VACANT              WHITE


Create a dataframe for the cities in Wyoming:

In [13]:
q = sdf_target['ST'] == 'WY'
left = sdf_target[q].copy()
left.head()

,index,AGE_18_21,AGE_22_29,AGE_30_39,AGE_40_49,AGE_50_64,AGE_5_17,AGE_65_UP,AGE_UNDER5,AMERI_ES,...,PLACEFIP,POP2000,POP2007,POP_CLASS,RENTER_OCC,SHAPE,ST,STFIPS,VACANT,WHITE
711,711,715,980,1527,2384,1765,2855,769,813,160,...,33740,11808,12183,6,1008,"{'x': -109.46492712301152, 'y': 41.51419117328...",WY,56,249,10879
712,712,1348,1963,2538,3299,2627,3797,1857,1279,160,...,67235,18708,19318,6,2074,"{'x': -109.22240010498797, 'y': 41.59092714080...",WY,56,1011,17164
715,715,700,1106,1648,2006,1381,2840,826,1000,122,...,25620,11507,11785,6,1253,"{'x': -110.96461812552366, 'y': 41.26330015271...",WY,56,607,10620
764,764,5469,5662,2966,3267,2879,3382,2191,1388,241,...,45050,27204,28111,6,5957,"{'x': -105.58725462620347, 'y': 41.31292665660...",WY,56,658,24704
766,766,2597,5697,7904,8204,8083,9791,7313,3422,430,...,13900,53011,54750,7,7585,"{'x': -104.80204559586696, 'y': 41.14554516058...",WY,56,1458,46707


Create a dataframe for the state of Wyoming:

In [14]:
q = sdf_join.STATE_ABBR == 'WY'
right = sdf_join[q].copy()
right.head()

,index,AGE_18_21,AGE_22_29,AGE_30_39,AGE_40_49,AGE_50_64,AGE_5_17,AGE_65_UP,AGE_UNDER5,AMERI_ES,...,POP2010,RENTER_OCC,SHAPE,SQMI,STATE_ABBR,STATE_FIPS,STATE_NAME,SUB_REGION,VACANT,WHITE
6,6,31070,48942,66252,82984,77968,97933,57693,30940,11133,...,548154,58094,"{'rings': (((-104.05361517875079, 41.698218253...",97813,WY,56,Wyoming,Mountain,30246,454670


Perform the spatial join:

In [16]:
from arcgis.features._data.geodataset.tools import spatial_join

In [17]:
sdf2 = spatial_join(df1=left, df2=right)
sdf2.head()

,TARGET_OID,JOIN_OID,index_left,AGE_18_21_left,AGE_22_29_left,AGE_30_39_left,AGE_40_49_left,AGE_50_64_left,AGE_5_17_left,AGE_65_UP_left,...,POP2000_right,POP2010,RENTER_OCC_right,SQMI,STATE_ABBR,STATE_FIPS,STATE_NAME,SUB_REGION,VACANT_right,WHITE_right
0,711,6,711,715,980,1527,2384,1765,2855,769,...,493782,548154,58094,97813,WY,56,Wyoming,Mountain,30246,454670
1,712,6,712,1348,1963,2538,3299,2627,3797,1857,...,493782,548154,58094,97813,WY,56,Wyoming,Mountain,30246,454670
2,715,6,715,700,1106,1648,2006,1381,2840,826,...,493782,548154,58094,97813,WY,56,Wyoming,Mountain,30246,454670
3,764,6,764,5469,5662,2966,3267,2879,3382,2191,...,493782,548154,58094,97813,WY,56,Wyoming,Mountain,30246,454670
4,766,6,766,2597,5697,7904,8204,8083,9791,7313,...,493782,548154,58094,97813,WY,56,Wyoming,Mountain,30246,454670
